<a id='head'></a>
# Table of Contents
* [0. General classes](#funcs)
* [1. Simulation model](#sim)
* [2. Math models](#anas)
    * [2.1 (h,m) math model](#ana_hm)
    * [2.2 (h,m,n) math model](#ana_hmn)
    * [2.3 Dynamic model](#ana_dynamic)
        * [2.3.1 Checker & printer](#check&print)
        * [2.3.2 Tracer](#tracer)

<a id='funcs'></a>
# 0. General classes<sup>[head](#head)</sup>

In [5]:
import numpy as np
import pandas as pd

import random
import math
import matplotlib
import matplotlib.pyplot as plt
import subprocess
import sys, os, argparse
import pickle as pkl
import copy
import numbers

from collections import Counter, defaultdict
from itertools import product
from fractions import gcd
from ast import literal_eval

import logging
import time

# Распределения
from scipy.stats import binom, geom, poisson
from scipy import sparse
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import spsolve, bicgstab, eigs, svds, ArpackNoConvergence
from scipy.sparse.csgraph import connected_components

import importlib
from numpy.fft import fft, ifft

# Распараллеливание
import ctypes
import multiprocessing as mp

<a id='sim'></a>
# 1. Simulation model<sup>[head](#head)</sup>

In [14]:
class SimulationModel(Checker):
    def __init__(self, input_file="input.txt",
                 model_filename='./imm_dynamic',
                ):
        """
        :param input_file - название входного файла с параметрами эксперимента
        :type input_file - string
        
        :param model_filename - название исполняемого файла имитации
        :type model_filename - executable
        """
        self.input_file = input_file
        self.model_filename = model_filename
        
        if not os.path.isfile(os.path.join(input_file)):
            raise ValueError("File \"{}\" is absent".format(input_file))
        if not os.path.isfile(os.path.join(model_filename)):
            raise ValueError("File \"{}\" is absent".format(model_filename))
        
    def __call__(self):
        args = [self.input_file]
        output = subprocess.check_output([self.model_filename] + args)
        output = output.decode('utf8').split('\n')
        args_dict = {}
        for i in range(len(output) - 1):
            if (output[i] == "seed"):
                args_dict[output[i]] = int(output[i+1])
            elif (output[i] == "res_period"):
                args_dict[output[i]] = int(output[i+1])
            elif (output[i] == "arr_period"):
                args_dict[output[i]] = int(output[i+1])
            elif (output[i] == "batchfile_path"):
                args_dict[output[i]] = output[i+1]
            elif (output[i] == "delay_bound"):
                args_dict[output[i]] = int(output[i+1])
            elif (output[i] == "det_per"):
                args_dict[output[i]] = float(output[i+1])
            elif (output[i] == "ran_per"):
                args_dict[output[i]] = float(output[i+1])
            elif (output[i] == "window_size"):
                args_dict[output[i]] = int(output[i+1])
            elif (output[i] == "rand_tx_duration"):
                args_dict[output[i]] = float(output[i+1])
            elif (output[i] == "det_tx_duration"):
                args_dict[output[i]] = float(output[i+1])
            elif (output[i] == "mean_access_time"):
                args_dict[output[i]] = float(output[i+1])
            elif (output[i] == "max_batch_size"):
                args_dict[output[i]] = int(output[i+1])
            elif (output[i] == "random_on"):
                args_dict[output[i]] = int(output[i+1])
            elif (output[i] == "current_age"):
                args_dict[output[i]] = int(output[i+1])
            elif (output[i] == "current_size"):
                args_dict[output[i]] = int(output[i+1])
            elif (output[i] == "batch_index"):
                args_dict[output[i]] = int(output[i+1])    
            elif (output[i] == "PLR"):
                PLR = float(output[i+1])
                
        return args_dict, PLR

<a id='anas'></a>
# 2. Math models<sup>[head](#head)</sup>

<a id='ana_hm'></a>
## 2.1 (h,m) math model<sup>[head](#head)</sup>

In [1]:
def find_stationary_distribution(P, max_error=1e-9, iterative=False, multiplier=1000):
    assert (len(P.shape) == 2) & (P.shape[0] == P.shape[1])

    size = P.shape[0]
    P = P.tocoo()
    P = (P - sparse.eye(size)).transpose().tocsr()

    selected_column = random.randrange(size)
    upper_P = P[:selected_column]
    lower_P = P[selected_column + 1:]
    row = sparse.csr_matrix(np.ones(size))
    if selected_column == 0:
        P = sparse.vstack([row, lower_P])
    elif selected_column == size - 1:
        P = sparse.vstack([upper_P, row])
    else:
        P = sparse.vstack([upper_P, row, lower_P])

    b = np.zeros(size)
    b[selected_column] = multiplier
    P = P.tocsr()
    if iterative:
        x0 = np.array([multiplier / size] * size)       # Нулевое приближение
        x = bicgstab(P, b, x0=x0, tol=1e-10, maxiter=4000)[0]
    else:
        x = spsolve(P, b)
    x /= multiplier
    return x

class BurstyVbrOrderedModel_hm:
    string_id = 'TXOP'
    number_id = -1

    def __init__(self, parameters):
        self.initialize(parameters)

    def initialize(self, parameters):
        self.parameters  = copy.deepcopy(parameters)

        self.model_type  = self.parameters['model_type']
        self.arr_period  = self.parameters['arr_period']
        self.input_distr = copy.deepcopy(self.parameters['input_distr'])
        self.res_period  = self.parameters['res_period']
        self.block_size  = self.parameters['block_size']
        self.delay_bound = self.parameters['delay_bound']
        self.per         = self.parameters['per']

        if 'seed' not in parameters:
            self.parameters['seed'] = 1
        if 'output_flow' not in parameters:
            self.parameters['output_flow'] = False

        if 'max_error' not in parameters:
            self.parameters['max_error'] = 1e-9
        if 'iterative' not in parameters:
            self.parameters['iterative'] = False

        if 'check_time' not in parameters:
            self.parameters['check_time'] = False

        self.seed = self.parameters['seed']
        self.output_flow = self.parameters['output_flow']

        self.max_error = self.parameters['max_error']
        self.iterative = self.parameters['iterative']

        self.check_time = self.parameters['check_time']

    def __call__(self):
        slot = gcd(self.arr_period, self.res_period)
        tres = self.res_period // slot
        tin = self.arr_period // slot
        d = self.delay_bound // slot
        input_probs = self.input_distr
        M = len(input_probs) - 1
        batch_sizes = np.array(range(M + 1))
        size = (d + tin + 1) * M
        forward_transform = lambda h, m: (h + tin) * M + m - 1

        ###################################################################

        start = time.clock()

        # Нахождение матрицы A
        self.selected_column = -1
        self.nonzeros = []
        self.row_indices = []
        self.col_indices = []
        
        for initial_h in range(-tin, d + 1):
            for initial_m in range(1, M + 1):
                initial_state = forward_transform(initial_h, initial_m)
                if initial_h < 0:
                    final_h = initial_h
                    final_m = initial_m
                    final_state = forward_transform(final_h, final_m)
                    self._add_transition(1, initial_state, final_state)
                else:
                    # Неуспешная передача
                    final_h = initial_h
                    final_m = initial_m
                    final_state = forward_transform(final_h, final_m)
                    self._add_transition(self.per, initial_state, final_state)

                    if initial_m == 1:
                        # Успешная передача при m = 1
                        final_h = initial_h - tin
                        for final_m in range(1, M + 1):
                            final_state = forward_transform(final_h, final_m)
                            self._add_transition((1 - self.per) * input_probs[final_m],
                                                 initial_state, final_state)
                    else:
                        # Успешная передача при m > 1
                        final_h = initial_h
                        final_m = initial_m - 1
                        final_state = forward_transform(final_h, final_m)
                        self._add_transition(1 - self.per, initial_state, final_state)

        A = sparse.csr_matrix((self.nonzeros, (self.row_indices, self.col_indices)),
                              shape=(size, size))

        # Нахождение матрицы C
        self.selected_column = -1
        self.nonzeros = []
        self.row_indices = []
        self.col_indices = []

        for initial_h in range(-tin, d + 1):
            for initial_m in range(1, M + 1):
                initial_state = forward_transform(initial_h, initial_m)
                if initial_h <= d - tres:
                    final_h = initial_h + tres
                    final_m = initial_m
                    final_state = forward_transform(final_h, final_m)
                    self._add_transition(1, initial_state, final_state)
                else:
                    n_lost = np.ceil(1.0 * max(0, initial_h + tres - tin - d) / tin)
                    final_h = initial_h + tres - tin - n_lost * tin
                    for final_m in range(1, M + 1):
                        final_state = forward_transform(final_h, final_m)
                        self._add_transition(input_probs[final_m],
                                             initial_state, final_state)

        C = sparse.csr_matrix((self.nonzeros, (self.row_indices, self.col_indices)),
                              shape=(size, size))
        A = A ** self.block_size
        P = C * A
        
        end = time.clock()
        if self.check_time:
            print('Time: building transition matrix:', end - start)

        ###################################################################

        start = time.clock()
        x = find_stationary_distribution(P, max_error=self.max_error, iterative=self.iterative, multiplier=1000)
        self.x = x
        end = time.clock()
        if self.check_time:
            print('Time: seeking for stationary distributions:', end - start)

        ###################################################################

        discarded_intensity = 0
        average_input_batch_size = np.sum(np.arange(M + 1) * input_probs)
        for h in range(d - tres + 1, d + 1):
            for m in range(1, M + 1):
                state = forward_transform(h, m)
                n_lost = np.ceil(float(max(0, h + tres - tin - d)) / tin)
                discarded_intensity += x[state] * (m + n_lost * average_input_batch_size)
        discarded_intensity /= tres
        input_intensity = average_input_batch_size / tin
        self.plr = discarded_intensity / input_intensity

        if self.output_flow:
            self.calculate_output_flow()

        return self.plr

    def _add_transition(self, probability, initial_state, final_state):
        if final_state != self.selected_column:
            if probability > 0:
                self.nonzeros.append(probability)
                self.row_indices.append(initial_state)
                self.col_indices.append(final_state)
            assert probability >= 0, "Negative probability"
            


<a id='ana_hmn'></a>
## 2.2 (h,m,n) math model<sup>[head](#head)</sup>

In [2]:
def find_stationary_distribution(P, max_error=1e-9, iterative=False, multiplier=1000):
    assert (len(P.shape) == 2) & (P.shape[0] == P.shape[1])
        
    size = P.shape[0]
    P = P.tocoo()
    P = (P - sparse.eye(size)).transpose().tocsr()
    
    selected_column = random.randrange(size)
    upper_P = P[:selected_column]
    lower_P = P[selected_column + 1:]
    row = sparse.csr_matrix(np.ones(size))
    if selected_column == 0:
        P = sparse.vstack([row, lower_P])
    elif selected_column == size - 1:
        P = sparse.vstack([upper_P, row])
    else:
        P = sparse.vstack([upper_P, row, lower_P])
    
    b = np.zeros(size)
    b[selected_column] = multiplier
    P = P.tocsr()
    if iterative:
        x0 = np.array([multiplier / size] * size)       # Нулевое приближение
        x = bicgstab(P, b, x0=x0, tol=1e-10, maxiter=4000)[0]
    else:
        x = spsolve(P, b)
    x /= multiplier
    return x

class BurstyVbrOrderedModel_hmn:
    string_id = 'TXOP'
    number_id = -1
    
    def __init__(self, parameters):
        self.initialize(parameters)
    
    def initialize(self, parameters):
        self.parameters  = copy.deepcopy(parameters)
        
        self.model_type  = self.parameters['model_type']
        self.arr_period  = self.parameters['arr_period']
        self.input_distr = copy.deepcopy(self.parameters['input_distr'])
        self.res_period  = self.parameters['res_period']
        self.block_size  = self.parameters['block_size']
        self.delay_bound = self.parameters['delay_bound']
        self.per         = self.parameters['per']
        self.cond_probs  = self.parameters['cond_probs']
        
        if 'seed' not in parameters:
            self.parameters['seed'] = 1
        if 'output_flow' not in parameters:
            self.parameters['output_flow'] = False

        if 'max_error' not in parameters:
            self.parameters['max_error'] = 1e-9
        if 'iterative' not in parameters:
            self.parameters['iterative'] = False
            
        if 'check_time' not in parameters:
            self.parameters['check_time'] = False
            
        self.seed = self.parameters['seed']
        self.output_flow = self.parameters['output_flow']
        
        self.max_error = self.parameters['max_error']
        self.iterative = self.parameters['iterative']
        
        self.check_time = self.parameters['check_time']
        
    def __call__(self, dim=3):
        slot = gcd(self.arr_period, self.res_period)
        tres = self.res_period // slot
        tin = self.arr_period // slot
        d = self.delay_bound // slot
        input_probs = self.input_distr
        cond_probs = self.cond_probs
        M = len(input_probs) - 1
        batch_sizes = np.array(range(M + 1))
        if dim == 3:               
            size = (d + tin + 1) * M ** 2
            forward_transform = lambda h, m, n: (h + tin) * M ** 2 + (m - 1) * M + n - 1                
        elif dim == 2:
            size = (d + tin + 1) * M                                         
            forward_transform = lambda h, m, n: (h + tin) * M + m - 1 + 0 * n
        else:
            assert False
        ###################################################################
        
        start = time.clock()
        
        # Нахождение матрицы A
        self.selected_column = -1
        self.nonzeros = []
        self.row_indices = []
        self.col_indices = []
        
        for initial_h in range(-tin, d + 1):
            for initial_m in range(1, M + 1):
                for initial_n in range(1, M + 1):
                    initial_state = forward_transform(initial_h, initial_m, initial_n)
                    if initial_h < 0:
                        final_h = initial_h                                                   
                        final_m = initial_m                                                         
                        final_n = initial_n
                        final_state = forward_transform(final_h, final_m, final_n)
                        self._add_transition(1, initial_state, final_state)
                    else:
                        # Неуспешная передача
                        final_h = initial_h
                        final_m = initial_m
                        final_n = initial_n
                        final_state = forward_transform(final_h, final_m, final_n)
                        self._add_transition(self.per, initial_state, final_state)
                        
                        if initial_m == 1:
                            # Успешная передача при m = 1
                            final_h = initial_h - tin
                            for final_m in range(1, M + 1):
                                final_n = final_m
                                final_state = forward_transform(final_h, final_m, final_n)
                                self._add_transition((1 - self.per) * cond_probs[initial_n, final_n],
                                                     initial_state, final_state)
                        else:
                            # Успешная передача при m > 1
                            final_h = initial_h
                            final_m = initial_m - 1
                            final_n = initial_n
                            final_state = forward_transform(final_h, final_m, final_n)
                            self._add_transition(1 - self.per, initial_state, final_state)
        
        A = sparse.csr_matrix((self.nonzeros, (self.row_indices, self.col_indices)),
                              shape=(size, size))
        
        # Нахождение матрицы C
        self.selected_column = -1
        self.nonzeros = []
        self.row_indices = []
        self.col_indices = []
        for initial_h in range(-tin, d + 1):
            for initial_m in range(1, M + 1):
                for initial_n in range(1, M + 1):
                    initial_state = forward_transform(initial_h, initial_m, initial_n)
                    if initial_h <= d - tres:
                        final_h = initial_h + tres
                        final_m = initial_m
                        final_n = initial_n
                        final_state = forward_transform(final_h, final_m, final_n)
                        self._add_transition(1, initial_state, final_state)
                    else:
                        n_lost = np.ceil(1.0 * max(0, initial_h + tres - tin - d) / tin)
                        assert n_lost == 0
                        final_h = initial_h + tres - tin - n_lost * tin
                        for final_m in range(1, M + 1):
                            final_n = final_m
                            final_state = forward_transform(final_h, final_m, final_n)
                            self._add_transition(cond_probs[initial_n , final_n],
                                                 initial_state, final_state)

        C = sparse.csr_matrix((self.nonzeros, (self.row_indices, self.col_indices)),
                              shape=(size, size))
        
        if dim == 2:
            A = A / M
            C = C / M
        
        A = A ** self.block_size
        P = C * A

        end = time.clock()
        if self.check_time:
            print('Time: building transition matrix:', end - start)
            
        ###################################################################
        
        start = time.clock()
        x = find_stationary_distribution(P, max_error=1e-9, iterative=False, multiplier=1000)
        self.x = x
        #print(x)
        end = time.clock()
        if self.check_time:
            print('Time: seeking for stationary distributions:', end - start)
        
        ###################################################################
        
        discarded_intensity = 0
        average_input_batch_size = np.sum(np.arange(M + 1) * input_probs)
        for h in range(d - tres + 1, d + 1):
            for m in range(1, M + 1):
                for n in range(1, M + 1):
                    state = forward_transform(h, m, n)
                    n_lost = np.ceil(float(max(0, h + tres - tin - d)) / tin)
                    assert n_lost == 0
                    discarded_intensity += x[state] * (m + n_lost * average_input_batch_size)    
        discarded_intensity /= tres
        input_intensity = average_input_batch_size / tin
        self.plr = discarded_intensity / input_intensity
        
        
        return self.plr
        
    def _add_transition(self, probability, initial_state, final_state):
        if final_state != self.selected_column:
            if probability > 0:
                self.nonzeros.append(probability)
                self.row_indices.append(initial_state)
                self.col_indices.append(final_state)
            assert probability >= 0, "Negative probability"

<a id='ana_dynamic'></a>
## 2.3 Dynamic model<sup>[head](#head)</sup>

<a id='check&print'></a>
### 2.3.1 Checker & printer<sup>[head](#head)</sup>

In [7]:
class Printer:
    def __init__(self, verbose, owner):
        """
        Класс, ответственный за распечатку сообщений. 
        Печатает сообщение, когда его уровень печати превосходит уноверь печати
        объекта класса.
        """
        self.verbose = verbose
        self.owner = owner
    def __call__(self, *args, **kwargs):
        if self.owner.verbose >= self.verbose:
            print(*args, **kwargs)


class Checker(Printer):
    def __init__(self):
        self.printers = {}
        for v in range(20):
            self.printers[v] = Printer(v, self) 
            
    def _check_numeric(self, n, name):
        if not isinstance(n, numbers.Number):
            raise TypeError("Param \"{}\" must be a number".format(name))
        return True 
    def _check_int(self, n, name):
        self._check_numeric(n, name)
        if not n == int(n):
            raise TypeError("Param \"{}\" must be an integer number".format(name))
        return True
    def _check_positive(self, n, name):
        self._check_numeric(n, name)
        if n <= 0:
            raise ValueError("Param \"{}\" must be positive".format(name))
        return True
    def _check_nonnegative(self, n, name):
        self._check_numeric(n, name)
        if n < 0:
            raise ValueError("Param \"{}\" must be nonegative".format(name))
        return True   
    def _check_proba(self, n, name):
        self._check_numeric(n, name)
        if (n <= 1) & (n >= 0):
            return True
        raise ValueError("Param \"{}\" is not proba though it must be".format(name))
    def _check_distr(self, distr, name):
        for i in range(len(distr)):
            self._check_proba(distr[i], name + '[{}]'.format(i))
        if not np.allclose(np.sum(distr), 1):
            raise ValueError("Distribution \"{}\" is not normed: sum equals {}".format(name, np.sum(distr)))
        return True

<a id='tracer'></a>
### 2.3.2 Tracer<sup>[head](#head)</sup>

In [10]:
class BaseTracer(Checker):
    TR_INFO_0 = 4
    TR_INFO_1 = 5
    TR_INFO_2 = 6
    TR_INFO_3 = 7
    
    TRACE_CONTINUES = 0
    TRACE_DIST1 = 1
    TRACE_DIST2 = 2
    TRACE_FINISHED = 3
    
    def next_batch_params(self, state):
        next_states_dict = {}
        r, h, m, n = state
        next_n = n + 1
        if r == self.TRACE_CONTINUES: #в этом состоянии trace известен
            init_n = self.get_init_index()
            if next_n - init_n == len(self.trace):
                if self.input_distr_dict is None:
                    next_states_dict[(self.TRACE_FINISHED, 0, next_n)] = 1.0
                else:
                    for batch_size, prob in self.input_distr_dict.items():
                        next_states_dict[(self.TRACE_DIST1, batch_size, -1)] = prob
            else:
                print(next_n - init_n, next_n)
                print(self.trace, len(self.trace))
                next_states_dict[(self.TRACE_CONTINUES, self.trace[next_n - init_n], next_n)] = 1.0
            return next_states_dict
        
        if r == self.TRACE_DIST1: #дальше по распределению
            for batch_size, prob in input_distr_dict.items():
                next_states_dict[(self.TRACE_DIST1, batch_size, next_n)] = prob
            return next_states_dict
        
        if r == self.TRACE_FINISHED: #всё передалось
            next_states_dict[(self.TRACE_FINISHED, 0, next_n)] = 1.0
            return next_states_dict

        assert False, 'Pain!'
        
    def get_init_index(self):
        assert len(self.statesprob_init) == 1, "ONLY ONE STATE IS ALLOWED"
        for i,j in self.statesprob_init.items():
            if i[0] == self.TRACE_CONTINUES:
                return i[3]
            else:
                assert False, 'WTF'
            
    
    def __init__(self, Tin, Tres, D, PER):
        """
        :param Tin - период поступления
        :type Tin - int
        
        :param Tres - период резервирования
        :type Tin - int
        
        :param D - delay bound
        :type D - int
        
        :param PER - вероятность ошибки
        :type PER - float
        """
        
        self._check_numeric(Tin, 'Tin')
        self._check_numeric(Tres, 'Tres')
        self._check_numeric(D, 'D')

        self._check_int(Tin, 'Tin')
        self._check_int(D, 'D')
        self._check_int(Tres, 'Tres')
        
        self._check_positive(Tin, 'Tin')
        self._check_positive(Tres, 'Tres')
        self._check_positive(D, 'D')
        
        self._check_numeric(PER, 'PER')
        self._check_proba(PER, 'PER')
        
        self.Tin = Tin
        self.D = D
        self.Tres = Tres
        self.PER = PER
       
        self.printers = {}
        for v in range(20):
            self.printers[v] = Printer(v, self)          
        
        self.initialized = False
        
    def set_Tres(self, Tres):
        self._check_numeric(Tres, 'Tres')
        self._check_int(Tres, 'Tres')
        self._check_positive(Tres, 'Tres')     
        self.Tres = Tres
        
    def reset(self, trace, statesprob_init, stateslost_init, statesarrived_init, input_distr_dict=None):
        self.trace = trace
        self.statesprob = Counter()
        self.statesprob_init = copy.deepcopy(statesprob_init)
        self.statesprob.update(statesprob_init)
        
        self.stateslost = Counter()
        self.stateslost.update(stateslost_init)
        self.stateslost_init = copy.deepcopy(stateslost_init)
        
        self.statesarrived = Counter()
        self.statesarrived.update(statesarrived_init)
        self.statesarrived_init = copy.deepcopy(statesarrived_init)
        
        self.input_distr_dict = copy.deepcopy(input_distr_dict)
        self.n_iters = 0
        self.initialized = True
    
    def __call__(self, W, min_tracked_proba=1e-8, verbose=-1, report_period=10000000, time_verbose=False):
        """
        Внимание:
        Изначально окно W начинается в момент времени 0, в котором также начинается некоторый зарезерированный 
        интервал. При повторных запусках окно W отсчитывается с резервирования, на котором 
        остановились предшествующие вычисления. Начальные offset-ыможно моделировать задав соответствующие
        problist и lostlist.
        
        Просчитываются все резервирования, начинающиеся в интервале[0, W]. 
        Если резервироваание начинается далее, то оно не рассматривается. Последующие 
        вычисления начинаются именно с него.
        
        При подсчете PLR число полученных пачек считается от 0-го момента, т.е. те, 
        что уже были в очереди, не в счет
        
        :param W - размер окна в тех же единицах, что и Tin, Tres
        :type W - int, float
        
        :param trace - трейс размеров пачек
        :type trace - numpy.array
        
        :param problist - вероятности начальных состояний
        :type problist - dict
        
        :param lostlist - потери начальных состояний
        :type lostlist - dict
        
        :param statesarrived - число поступивших пакетов
        :type statesarrived - dict
        
        :param input_distr - распределение пачек входного потока
        :type input_distr - numpy.array размера M + 1, где M - максимальный размер пачки

        :param verbose - уровень печати вспомогательных сообщений
        :type verbose - int
        
        :param report_period - период печати статистики
        :type reprot_period - int
        """
        
        if not self.initialized:
            assert False, "not initialized"
        self.verbose = verbose
        self.min_tracked_proba = min_tracked_proba
        
        Nres = int(float(W) / self.Tres) + 1
        self.printers[self.TR_INFO_0]("Anticipated number of iterations equals {}".format(Nres))
        
        from tqdm import tqdm
        generator = range(Nres)
        if time_verbose:
            generator = tqdm(generator)
            
        for nres in generator:
            self.nres = nres
            self.n_iters += 1
            if (nres + 1) % report_period == 0:
                plr = self.get_plr(throw=False)
                self.printers[self.TR_INFO_0](
                    'Number if iteration {}, state space size equals {}, curr plr = {}'.format(
                    nres + 1, len(self.statesprob), plr))
            
            """plrs.append(self.get_plr(throw=False)) # DEBUGGGGGG
            if plrs[-1] > 0.01:
                print('PLR EXCEEDED TRESHOLD')
                self.verbose = 10
                r_iter = 20
            if r_iter < 0:
                return 0"""
                
            self.next_statesprob = Counter()
            self.next_stateslost = Counter()
            self.next_statesarrived = Counter()
            
            for state, prob in self.statesprob.items():
                self.printers[self.TR_INFO_1]("\tTransitions from state {} with probability {}".format(
                    state, prob))

                losts = self.stateslost[state]
                arrived = self.statesarrived[state]
                r, h, m, n = state
                
                if m == 0:
                    self.printers[self.TR_INFO_2]("\t\tState {} is final state".format(state))
                    # Достигнуто финальное состояние, так как трейс закончился. Остаемся в том же состоянии.
                    self._add_proba(state, state, 1.0, losts, arrived)
                    continue
                if h < 0:
                    self.printers[self.TR_INFO_2]("\t\th = {} is negative".format(h))
                    next_state = (r, h + self.Tres, m, n)
                    self._add_proba(state, next_state, 1.0, losts, arrived)
                elif h <= self.D - self.Tres:
                    self.printers[self.TR_INFO_2]("\t\th = {} is in range [0, D - Tres]".format(state))
                    # Успешная передача
                    if m == 1:
                        # Размер пачки равен 1 - смена пачки
                        next_h = h + self.Tres - self.Tin
                        for (next_r, next_m, next_n), prob in self.next_batch_params(state).items():
                            next_state = (next_r, next_h, next_m, next_n)
                            tr_prob = (1 - self.PER) * prob
                            self._add_proba(state, next_state, tr_prob, losts, arrived + 1) # +next_m
                    else:
                        # Размер пачки больше 1 - уменьшение пачки
                        next_state = (r, h + self.Tres, m - 1, n)
                        tr_prob = 1 - self.PER
                        self._add_proba(state, next_state, tr_prob, losts, arrived + 1) # + 0

                    # Неуспешная передача
                    next_state = (r, h + self.Tres, m, n)
                    tr_prob = self.PER
                    self._add_proba(state, next_state, tr_prob, losts, arrived)
                else:
                    self.printers[self.TR_INFO_2]("\t\th = {} is higher than D - Tres".format(state))
                    next_h = h + self.Tres - self.Tin
                    for (next_r, next_m, next_n), prob in self.next_batch_params(state).items():
                        next_state = (next_r, next_h, next_m, next_n)
                        self._add_proba(state, next_state, prob, losts + m - 1 + self.PER, arrived + m) # + next_m
                        
            # Перенормировка
            s = sum(self.next_statesprob.values())
            assert np.allclose(s, 1, atol=10e-04), "Sum of probabilities equals {}".format(s)

            for state, prob in self.next_statesprob.items():
                self.next_statesprob[state] /= s
                self.next_stateslost[state] /= prob
                self.next_statesarrived[state] /= prob
                
            self.statesprob = self.next_statesprob
            self.stateslost = self.next_stateslost
            self.statesarrived = self.next_statesarrived
        #print('N_ITERS = {}'.format(self.n_iters))
        # Время подвести итоги
        plr = self.get_plr()
        return plr
    
    def get_plr(self, throw=True):
        PLR = 0
        for state, prob in self.statesprob.items():
            if (self.statesarrived[state] == 0) & throw:
                raise ValueError("No packets arrived during transition to state {}." \
                                 "Please, provide longer window.".format(state))
            PLR += prob * self.stateslost[state] / max(self.statesarrived[state], 1)
        return PLR
    
    def _add_proba(self, state, next_state, tr_prob, losts, arrived):
        prob = self.statesprob[state]
        p = prob * tr_prob
        if p >= self.min_tracked_proba:
            self.next_statesprob[next_state] += p
            self.next_stateslost[next_state] += p * losts
            self.next_statesarrived[next_state] += p * arrived
            self.printers[self.TR_INFO_3]("\t\t\tTransition to {} with proba {}".format(next_state, tr_prob))

    def print_probas(self):
        self._print_counter(self.statesprob)
    def print_losts(self):
        self._print_counter(self.stateslost)
    def print_arrived(self):
        self._print_counter(self.statesarrived)
        
    def _print_counter(self, d):
        for state, value in sorted(list(d.items()), key=lambda x: x[0]):
            print('\t', state, value)